In [27]:
import pandas as pd, numpy as np

In [28]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
subm = pd.read_csv('input/sample_submission.csv')

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [29]:
train[COMMENT] = train[COMMENT].str.lower()
test[COMMENT] = test[COMMENT].str.lower()

In [30]:
train[COMMENT] = train[COMMENT].str.replace('https?:\/\/[^\s]*','')
test[COMMENT] = test[COMMENT].str.replace('https?:\/\/[^\s]*','')
train[COMMENT] = train[COMMENT].str.replace('=','')
test[COMMENT] = test[COMMENT].str.replace('=','')

In [31]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"nonsense? kiss off, geek. what i said is true...",1,0,0,0,0,0
1,27450690,"""\n\n please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"points of interest"""" \n\ni removed the...",0,0,0,0,0,0
3,77493077,asking some his nationality is a racial offenc...,0,0,0,0,0,0
4,79357270,the reader here is not going by my say so for ...,0,0,0,0,0,0


In [32]:
train[train['severe_toxic']==1]['comment_text']

213      f'uck you administrators (i will be back to tr...
480      ah haahhahahah yaaa bitch yaaaa!@!!!!!!ah haah...
721      frozen sucks lion king is the bestfrozen sucks...
811                      this user is a fucking douche bag
869      fuck  \n\n62.158.73.165  62.158.73.165  62.158...
1223     fuck you \n\nyou suck. leave me alone. fuck ti...
1563     i am going to shoot you in the head and laugh ...
1695     you're a dirty faggot \n\nlick my hairy nut sa...
1703     stop being a shithead, or i will rip off your ...
1744     and why the fuck you say fuck you up there? fu...
1916     fuck wikipedia and fuck all the leftist, liber...
1951     stop reverting my edit\nyour so fucken gay get...
2030     securityfuck homeland securityfuck homeland se...
2263     are you a fucking dick-head?  people like you ...
2484     fuck you orange suede sof \n\nfuck you orange ...
2492     hello \n\ngo fuck yourself, kid.  adults are t...
2592     fuck you you stupid and gay bastard who thinls.

Prepare FastText training data

In [33]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['clean'] = 1-train[label_cols].max(axis=1)
train.describe()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
count,9.585100e+04,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000
mean,4.994359e+11,0.096368,0.010068,0.053301,0.003182,0.049713,0.008492,0.897862
std,2.890136e+11,0.295097,0.099832,0.224635,0.056320,0.217352,0.091762,0.302831
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.001297e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.501088e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999882e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
from sklearn.model_selection import train_test_split
for label in label_cols:    
    train['label'] = ""
    train.loc[(train[label]==1), 'label'] = "__label__" + label + " "
    train.loc[(train[label]==0), 'label'] = "__label__clean" + " "
    #train_set, val_set = train_test_split(train, test_size=0.2, random_state=1)
    #train_set.to_csv("fasttext_train_"+label+".csv", columns=['label', 'comment_text'], index=False)
    #val_set.to_csv("fasttext_val_"+label+".csv", columns=['label', 'comment_text'], index=False)
    train.to_csv("fasttext_train_"+label+".csv", columns=['label', 'comment_text'], index=False)

In [35]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean,label
0,22256635,"nonsense? kiss off, geek. what i said is true...",1,0,0,0,0,0,0,__label__clean
1,27450690,"""\n\n please do not vandalize pages, as you di...",0,0,0,0,0,0,1,__label__clean
2,54037174,"""\n\n """"points of interest"""" \n\ni removed the...",0,0,0,0,0,0,1,__label__clean
3,77493077,asking some his nationality is a racial offenc...,0,0,0,0,0,0,1,__label__clean
4,79357270,the reader here is not going by my say so for ...,0,0,0,0,0,0,1,__label__clean


# Training
~/fb_research/fasttext/fastText-0.1.0/fasttext supervised -input  fasttext_train_identity_hate.csv -output identity_hate -minn 3 -maxn 6 -loss hs
~/fb_research/fasttext/fastText-0.1.0/fasttext supervised -input  fasttext_train_insult.csv -output insult -minn 3 -maxn 6 -loss hs
~/fb_research/fasttext/fastText-0.1.0/fasttext supervised -input  fasttext_train_obscene.csv -output obscene -minn 3 -maxn 6 -loss hs
~/fb_research/fasttext/fastText-0.1.0/fasttext supervised -input  fasttext_train_severe_toxic.csv -output severe_toxic -minn 3 -maxn 6 -loss hs
~/fb_research/fasttext/fastText-0.1.0/fasttext supervised -input  fasttext_train_threat.csv -output threat -minn 3 -maxn 6 -loss hs
~/fb_research/fasttext/fastText-0.1.0/fasttext supervised -input  fasttext_train_toxic.csv -output toxic -minn 3 -maxn 6 -loss hs

~/fb_research/fasttext/fastText-0.1.0/fasttext test identity_hate.bin fasttext_val_identity_hate.csv
~/fb_research/fasttext/fastText-0.1.0/fasttext test insult.bin fasttext_val_insult.csv
~/fb_research/fasttext/fastText-0.1.0/fasttext test obscene.bin fasttext_val_obscene.csv
~/fb_research/fasttext/fastText-0.1.0/fasttext test severe_toxic.bin fasttext_val_severe_toxic.csv
~/fb_research/fasttext/fastText-0.1.0/fasttext test threat.bin fasttext_val_threat.csv
~/fb_research/fasttext/fastText-0.1.0/fasttext test toxic.bin fasttext_val_toxic.csv


In [11]:
from pyfasttext import FastText

In [31]:
# https://github.com/facebookresearch/fastText/blob/master/docs/supervised-tutorial.md
# model = FastText()
# model.supervised(input='fasttext_train.csv', output='fasttext_model', epoch=500, lr=0.7)

In [36]:
models = {}
for label in label_cols:
    models[label] = FastText(label+'.bin')
    print(models[label].labels)

['clean', 'toxic']
['clean', 'severe_toxic']
['clean', 'obscene']
['clean', 'threat']
['clean', 'insult']
['clean', 'identity_hate']


In [13]:
models['toxic'] = FastText('toxic.bin')

In [76]:
models['toxic'].predict_proba(["hello how are you"], k=2)
#models['insult'].predict_proba(["you"], k=2)

[[('insult', 1.0), ('clean', 1.9531265169053625e-08)]]

# predict

In [37]:
comment_ids = []
comments = []
for index, row in test.iterrows():
    comments.append(row['comment_text'])
    comment_ids.append(row['id'])

predict_probas = {}
for label in label_cols:
    predict_probas[label] = models[label].predict_proba(comments,k=2)   

In [54]:
import csv
with open('submit.csv', "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])
    csv_rows = []
    for index, comment_id in enumerate(comment_ids):
        #print('processing',index)
        prob_dict = {}        
        #print(comments[index])
        for label in label_cols:       
            #print(predict_probas[label][index])            
            for label_predict, prob in predict_probas[label][index]:                
                if label_predict != 'clean':    
                    prob_dict[label] = prob
        csv_row=[]
        csv_row.append(comment_id)
        
        for label in label_cols:
            if label in prob_dict:
                csv_row.append(prob_dict[label])
            else:
                csv_row.append(0)
                #print('no prediction:',index,'comment:',comments[index])
                #csv_row.append(prob_dict[label])
        csv_rows.append(csv_row)    
    writer.writerows(csv_rows)

In [22]:
pred = pd.read_csv('submit.csv')

In [23]:
pred.loc[pred['severe_toxic'] > 0.8]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
2073,9152726631,1.000000,0.898438,1.000000,4.296877e-02,9.902344e-01,0.550781
2291,10093563914,0.001953,0.998047,0.162109,3.417969e-01,1.367189e-02,0.955078
2991,13207123035,1.000000,0.857422,1.000000,2.636719e-01,9.980469e-01,0.210938
3221,14282370093,1.000000,0.859375,1.000000,5.878906e-01,1.000000e+00,0.396484
5727,25308734743,1.000000,0.960937,1.000000,6.445314e-02,1.000000e+00,0.267578
6037,26465568197,1.000000,0.894531,1.000000,2.148440e-02,3.964844e-01,0.062500
6427,28018151504,1.000000,0.960937,1.000000,1.093750e-01,9.980469e-01,0.197266
6507,28410502015,1.000000,0.890625,1.000000,1.953127e-08,9.570314e-02,0.001953
6551,28661663728,1.000000,0.902344,1.000000,5.664065e-02,1.000000e+00,0.730469
7897,34528220435,1.000000,0.947266,1.000000,1.894531e-01,1.000000e+00,0.728516


In [118]:
test.loc[test['id'] == 787050824879]['comment_text']

201691    == credit ==
Name: comment_text, dtype: object